In [9]:
import cv2
from tensorflow import keras
import numpy as np
def convert_video_to_frames(videoPath):
    frames = []
    capture = cv2.VideoCapture(videoPath)
    while True:
        ret, frame = capture.read()
        if not ret:
            break
        frames.append(frame)
    capture.release()

    return frames

def test_frame(frame, model):

    resized_frame = cv2.resize(frame, (224, 224))
    normalized_frame = resized_frame.astype('float32') / 255.0
    prediction = model.predict(np.expand_dims(normalized_frame, axis=0))
    predicted_label = prediction.argmax()
    confidence_value = prediction[0, predicted_label]

    return classification, confidence_value

def write_to_video(frame, classification, confidence_value, video_writer):
    text = f"Classification: {classification}, Confidence: {confidence_value:.2f}"
    cv2.putText(frame, text, (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    video_writer.write(frame)

video_path = "input_video.mp4"
frames = convert_video_to_frames(video_path)

model = keras.models.load_model("drowsiness_detection_model.h5")

classifications = []
confidence_values = []
for frame in frames:
    classification, confidence_value = test_frame(frame, model)
    classifications.append(classification)
    confidence_values.append(confidence_value)

video_writer = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frames[0].shape[1], frames[0].shape[0]))

for i in range(len(frames)):
    write_to_video(frames[i], classifications[i], confidence_values[i], video_writer)

video_writer.release()

cv2.startWindowThread()
cap = cv2.VideoCapture("output.mp4")
while True:
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow("Output Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

1/1 [==============================] - 0s 15ms/step


In [ ]:
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()